# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [3]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # 模擬網頁下拉
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待一段時間讓網頁更新
    time.sleep(SCROLL_PAUSE_TIME)

    # 檢視目前網頁底端
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中...")

    

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [5]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [6]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 新台幣開盤升值3.1分　以29.780元開出

▲新台幣開盤升值3.1分。（圖／記者湯興漢攝）
記者陳心怡／台北報導
台北外匯市場新台幣兌美元匯率今（9）日開盤升值3.1分，以29.780元兌1美元開出。
新台幣匯率昨（8）日收在29.811元兌1美元，今天以29.780元兌1美元開出、升值3.1分。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[1] 台股早盤下跌47點　指數跌破11600關卡

▲台股今日開盤下跌10點。（示意圖／記者湯興漢攝）
記者王曉敏／台北報導
美股三大指數周一（8日）收漲逾1%，標普500指數更完全收復年初以來失土。台股今（9）日開盤下跌10點，以 11600點開出。
截至上午9點04分，指數來到11562點，下跌47點。
蘋概三王中，台積電（2330）以316.5元開出，下跌0.47%；鴻海（2317）開在79.3元，下跌0.5%；股王大立光（3008）以4425元開出，上漲0.11%。
美股延續上周漲勢，科技股和通訊股的上漲推升那斯達克指數周一收盤創新高，自3月23日觸底以來已累計上漲了44.7%。標普500指數的年漲幅回歸正值，但與道瓊指數仍分別較歷史高點低4.5%及6.7%。
周一美股表現，道瓊指數上漲461.46點或1.7%，報27,572.44點；標普500指數上漲38.46點或1.2%，報3,232.39 點；那斯達克指數上漲110.66點或1.13%，報9,924.75 點；費城半導體指數下跌5.14點或0.26%，報1,998.62 點。
----------------------------------------------------------------------

[2] 影／51家業者推三倍券加碼回饋　信用卡祭特定方案最高上看30%

 記者林淑慧／台北報導
振興三倍券七月上路，經濟部加速彙整支付與信用卡等業者回饋方案，官員指出，除了台灣PAY祭出加碼回饋上看1000元的好康優惠外，信用卡業者也推出特定日期與店家

▲防疫、新興商機與中美貿易戰等因素，推升美國5月成為台灣出口的第二大市場。（圖／路透社）
記者吳靜君／台北報導
財政部今（8）日公布台灣5月海關進、出口貿易情形，受惠於中美貿易戰、遠距辦公以及新興商機等推波助瀾之下，我國對美5月的出口金額42.72億美元，睽違11年以來超越東協，成為我國出口第二大市場。
5月主要出口市場，呈現兩樣情；財政部公布，5月對陸港、美、日出口齊揚；對東協及歐洲則呈現縮減。
在對陸港出口方面，我國5月對其出口金額為121.2億美元，較去年同期增加11.4億美元、年增率10.3%，也創近10年新高。財政部分析，對大陸出口的依賴度增加，不單單是這個月，而是這一陣子以來都是如此，主要的出口項目仍是為電子零組件，主要因素是受到新興科技運用、5G和高效能運算的發展等，市場看好利基下升溫，較不易受疫情影響有所改變；另外5月也看到對陸、港出口中，光學器材持續增加，其出口重鎮也是中國大陸，推測可能與中國大陸積極復工、接到急單有關。
5月對美出口42.7億美元，創歷年單月新高，較去年同期增加3.6億美元、年增9.3%，也睽違11年出口值超越東協，成為第二大市場；主要出口項目包含資通訊產品等。財政部表示，主要是受惠於中美貿易戰、台商產能回流以及疫情所帶來的遠距商機，而美國陸續解封帶來的訂單效應等也有刺激作用。
就今年前5月來看，對中國大陸、香港與美國成為台灣出口相當重要的市場，對陸、港出口占比為41.5%，創十年來最高紀錄；對美國的出口占比14.5%，也有14年新高的紀錄。
電子零組件、資通訊商品　撐起我國出口半壁江山
出口的主要貨品當中，電子零組件與資通產品撐起出口的「半壁江山」，電子零組件5月出口金額達102.4億美元，繼續有百億元水準，年增12億美元、13.2%；資通訊產品出口39.5億美元，創歷史第三高紀錄，年增3.9億美元、10.9%。
財政部表示，就主要的國家當中，我國前5月的出口金額還有正成長1.5%，相較於南韓前5月-11.2%、中國大陸-7.7%、美國前4月-9.5%、日本-8.4%、香港-7.4%、新加坡-6.9%、德國-6.1%，算是「萬黑叢中的一點紅」了。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

▲DS4造型悠遊卡開放預購。（圖／記者邱倢芯攝）
記者楊亞璇／台北報導
由Sony授權製作、以DUALSHOCK®4無線控制器(簡稱「DS4」)為造型所設計的悠遊卡，8日上午11時再度開放預購，悠遊卡公司8日晚間說明，截至8日下午5時，各通路整體預售量約達18萬張，只要在6月8日至10日預購的消費者，一定買得到，不過依目前生產時程，可能需等到2021年下半年後才可取貨。
悠遊卡公司指出，經了解各通路銷售狀況，7-Eleven即將在9日0時5分重新開放預購，萊爾富預估將在9日凌晨0時重啟預購，全家便利商店則將在今天（8日）下午6時繼續開放預購，OK超商則正常預購中，電商通路PChome也將在9日凌晨0時開放預購，至於MoMo購物網則停止預購。
悠遊卡公司說明，此次預購保證「限時不限量」，不過受限於工廠產能及製作期程，各通路依照「取貨日」不同，陸續將商品上架，預購系統轉換需要時間，經與各通路溝通，已盡快進行系統調整，將後續取貨日之生產數量上架，確保消費者在期限內都能順利完成預購。
----------------------------------------------------------------------

[24] 連3黑！5月出口年減2%　財部：防疫商機部分抵銷負面因素

▲新冠肺炎仍具不確定性、中美科技戰再起，我下半年出口仍然審慎（圖／記者林世文攝）
記者吳靜君／台北報導
財政部今（8）日公布海關進出口貿易統計，5月出口為270億美元，較4月增加7%，較去年同期減少2%，連續三個月負成長，累計前5月出口為1309.1億美元，較去年同期增加1.5%。財政部表示，但是因為新冠肺炎（COVID-19）疫情，遠端辦公、視訊與宅經濟順勢興起和廠商回流支撐，抵銷部分的負面因素。
5月進口金額為222.8億美元，較4月減少3%，較去年同期減少3.5%，累計前5月進口1144億美元，較去年同期增加1.4%。5月出超的金額達到47.2億美元，較去年同期增加2.5億美元；累計前5月出超165.1億美元，較去年同期增加3億美元。
財政部表示，新冠肺炎疫情持續衝擊全球貿易活動，加上國際原物料價格走弱，我國傳產貨品外銷仍舊低迷，但是因為疫情因素，遠端辦公、視訊與宅經濟順勢興起，以及廠商回流擴增產能形成支撐，抵銷部分負面衝擊。
但是，展望6月新冠肺炎不確定因素仍然存在，不易擺脫

▲一名男大生表示自己進股市2個月就賺進1千萬元。（資料照／記者湯興漢攝）
記者陳心怡／台北報導
大環境不景氣，許多普通上班族可能一輩子都賺不到1000萬元，但是一名自稱20歲男大生，在Dcard發表《進股市人生賺的第一個1000萬》一文，聲稱進股市短短2個月時間，就把本金從3百多萬增值至1804萬，還貼出對帳單截圖，證明所言不虛，不少網友留言表示佩服，更羨慕男大生能有300萬的閒錢投資股市。
不過，《ETtoday新聞雲》向雲科大查證，該校公關組長表示，能在Dcard上面發文的不一定真的是大學生。該平台發言者均為匿名，除非有在留言處透露名字與學籍班級，否則無法證實其身份。之前曾有過許多帶風向哄投資的案例，因無更多線索，所以無法針對此事評論。
該位留言顯示為國立雲林科技大學男大生的網友，日前在Dcard發表《進股市人生賺的第一個1000萬》一文。他表示投資風險真的要管理好，發現風險：期貨>當沖>股票，當初用50萬買了原油。另外，他也分享心得：「永遠不要和川普對做」，台股則是「跟著主力做才會賺錢，但有時候可以先提前布局」。
根據男大生附的對帳單，他的總成本是313萬出頭，而目前股票市值達到1800多萬，報酬率高達472.97％。他還附上兩支股票中天、合一對帳單，投報率高達53.21％和46.75％。
▲男大生貼出自己的對帳單。（圖／翻攝自Dcard）
檢視合一（4743）的股價，從4月的30元左右，一路飆漲，現在股價已經來到176元；中天（4128）股價也從4月的13元左右起漲，目前已接近40元，兩檔股票確實都是上漲的。不過，這還是無法證明，該位網友真的有買進，所說的都是真的，畢竟截圖未必是真的。
許多網友紛紛回應：「厲害是厲害，但20歲就有300萬閒錢能拿來投資比較讓人羨慕…」、「20歲就擁有300萬資本真的不容易 難道這就是所謂的新手運」、「我比較好奇的是初始資金怎麼來的」、「高投資高報酬相對風險也大 建議任何投資前都先了解標的 最怕是盲目跟單無腦多 等到被倒貨怎麼套都不知道」。
►過期票券別浪費！簡單3步驟　教你搶救妙招
----------------------------------------------------------------------

[33] 財政部常務次長　賦稅署長李慶華正式升任

▲賦稅署署長李慶華正式升任財政部常次。（圖／記者吳

特約記者／陳冠宇報導
罷韓通過後引發許多討論，不過很多人都只注意短期分析，沒看到長遠影響。筆名喬麵的資深電訊專家指出，台灣政治會越來越惡化並邊緣化，想要確保台灣生存的最重要條件，就是強化台積電這座護國神山，這有助於台灣在爭取生存發展時左右逢源。
喬麵認為，經過這次罷韓的教訓，國民黨必須有更本土化的論述。他認為國民黨會先去中國化，再來就要看自己的造化，畢竟自1911年以來，這110年從未被消滅，所以要泡沫或被消失，當然也不會那麼容易；至於民進黨，要努力避免落入傲慢與貪腐的陷阱，而不該持續加速這個過程。喬麵分析指出：台灣政治未來變化的重點是民眾黨等第三本土政黨成不成材，如果能發揮改變的力量，就有助於良性發展，如果不能發揮改變的力量，恐怕台灣的政治環境會更加惡化而且在國際政治中被邊緣化。
喬麵從宏觀的角度分析指出：台灣現在最大的挑戰是如何在兩大強權的對抗中生存與強大，不能只有押寶美國而已，如何用智慧跟我們14億人口的「血緣」鄰居相處是個必須面對的挑戰。
具有數十年電訊產業高階管理經驗的喬麵認為，全球經濟產業鏈與國際政治的關係密不可分，有遠見的政府必須提供所有的資源，把半導體上下游產業全數以最優惠的租稅條件吸引回台，以保有台灣自我獨立生存的優勢。他甚至說出驚人之語：「台灣最終只剩下『台積電共和國』。」
----------------------------------------------------------------------

[48] 英鎊匯價創近3個月新高！　換新台幣5萬要多花3871元

記者陳心怡／台北報導
英鎊換匯價回到近3個月以來高點位置，根據台銀今（8）日上午牌告的外幣現金賣出匯價，英鎊匯價來到38.71 元，相較3月低點匯價，如今換新台幣5萬會少拿到100英鎊，等於損失了新台幣3871元。
根據據台銀今日上午牌告的現金賣出匯價，目前英鎊匯價為38.83元，來到今年3月13日（匯價38.83元）、近3個月以來的高價。
如要換新台幣5萬元，今天的匯價可以換到1,392英鎊；但如果是以3月19日的低點匯價35.91元來計算，當時可以換到1,291英鎊，足足就少了近100英鎊，等於口袋蒸發了新台幣3871元。
根據彭博即時匯率，英鎊目前來到1.2702美元兌1英鎊，為3月11日匯率1.2820美元兌1英鎊以來新高。
最近隨著市場對英國央行採取負利率的

▲宏達電執行長Yves Maitre推特發文。（圖／翻攝推特）
記者姚惠茹／台北報導
反種族歧視遊行浪潮席捲全球，宏達電（2498）執行長梅特爾（Yves Maitre）特別在推特發文力挺，內容提到對近來的悲劇感到無言，並震驚看見執法者侵害他人權利，並強調宏達電將透過科技授與的力量與教育來實現全人類更美好的生活。
美國黑人喬治‧佛洛伊德（George Floyd）被警員制伏期間死亡，引發美國首都華盛頓數以萬計的人參與反種族歧視遊行，並在紐約、芝加哥、舊金山等地也有群眾示威，這股浪潮甚至蔓延至英國、法國、德國、西班牙和澳大利亞。
Yves Maitre特別在推特發文談到，就像你們多數人一樣，自己對近來的悲劇感到無言，並震驚看見執法者侵害他人權利，因為沒有人擁有奪走生命的權利，並希望藉此鼓勵被害者家庭，以及那些同樣遭受迫害，但故事沒有被曝光的人們。
Yves Maitre指出，宏達電將透過科技授與的力量與教育來實現全人類更美好的生活，並希望人們知道，宏達電與大家同在，透過聆聽大家的聲音，宏達電承諾將會透過自己在相關產業的能力位置，為人類創造更好的生活環境。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[58] 實施負利率可能性低　英鎊兌美元創3月以來單週最佳表現

▲英鎊兌美元匯價持續攀升。（圖／達志影像／美聯社）
記者陳心怡／外電報導
在3月份時，由於新冠肺炎 (COVID-19) 疫情席捲歐洲，導致英鎊兌美元匯價跌至 1.14，但最近隨著市場對英國央行採取負利率的疑慮逐漸消除，促使英鎊兌美元匯價持續攀升，並創下自3月份以來最大的單週漲幅。
根據《CNBC》報導，美元一直以來都是種避險資產，但近日來，由於市場預期在經濟重啟後，全球景氣有望逐漸復甦，使得美元大幅貶值。此外，美國日前公佈的就業市場數據表現亮眼，5 月份的新增非農就業人數增加 250 萬人，遠低於原先預期的833萬人。經濟數據的改善也使得美元匯價持續下跌。
國際匯款服務平台 OFX 分析師 Sebastian Clements 表示，先前大規模失業潮、中美貿易戰及示威抗議活動等，加上各國所能採取的經濟刺激空間有限，使得投資人湧入美元避險。但近日來，投資人開始

▲左起為東典副董事長蘇立群、董事長陳正德。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
東典光電（6588）今（8）日宣布，配合初次上櫃前股票公開承銷，6月9日起至11日止舉行競價拍賣，底價為每股57.63元，總張數2,330張，每筆標單最低投標數量為1張，以價高者優先得標，全案將訂在6月15日上午10點開標，並預計在7月初正式掛牌上櫃。
東典光電表示，旗下主要生產光學薄膜濾光片，累積近20年生產經驗，產品種類多，包括高密度多工分波器(DWDM)、低密度多工分波器(CWDM)、增益平坦濾波器(GFF)、波段分波器(Band Separators Filters)、抗反射膜及截止濾光片等。
東典光電指出，迎接5G時代來臨，隨著傳輸距離變長，以及戶外基站對於寬溫要求提高，需要兼容多通道，韓國基站規格均以DWDM為主，中國大陸部分電信商前端採CWDM，後端採用DWDM，若以韓國今年5G基站30萬座計算，預估濾波片需求量約0.27億片，目前市場需求遠大於供給。
東典光電說明，目前共有14台鍍膜機，包含DWDM專用設備11台及其他特殊產品設備3台，產能方面，2019年全年度公司正常產能可達453萬片，最終產出367萬片，產能利用率約81%，因此擬計劃向專業廠商購置3台鍍膜機，今年底前會有1台進廠，另外2台將視疫情變化機動調整。
東典光電補充，5G通訊啟動，朝向資訊化生活布局，市場看好智慧製造、智慧交通、遠距醫療、居家看護機器人等發展，對未來光通訊濾片需求量大，光通訊薄膜濾光片產品種類繁多，規格要求難以統一。
東典光電強調，旗下擁有相當完整的產品線，以滿足客戶一次購足的需求，並因應客戶各項產品需求，提供客製化的完整服務，逐漸朝高階產品發展邁進，再加上今年第4季陸續有擴廠及採購新機台的規劃，可望再拉升產能，迎戰5G光通訊時代搶佔商機。
►過期票券別浪費！簡單3步驟　教你換1.5倍東森幣
----------------------------------------------------------------------

[69] 生技、永續類股超級夯　期交所再推2項新避險期貨商品

▲期交所再推2項新避險商品。（圖／記者余弦妙攝）
記者余弦妙／台北報導
臺灣期貨交易所(以下稱期交所)在109年6月8日上市「FTSE4GOOD臺灣指數公司臺灣永續指數期貨」(以下簡稱臺灣永續期

▲隨著國內疫情趨緩，政府開始推動國旅，帶動觀光產業，許多人迫不及待安排國內旅遊。（圖／記者賴文萱攝）
記者紀佳妘／台北報導
隨著新冠肺炎（COVID-19）疫情趨緩，民眾出門旅行意願大增，銀行力挺國旅商機，陸續推出信用卡高額回饋方案，包括訂房折扣、旅行社等優惠，刷卡回饋率上看14.7%。
根據萬事達卡消費者旅遊調查報告指出，在解封之後，有58%台灣民眾最想去花東或離島旅遊，且對於安排國內2至3天的旅遊行程的意願大幅增加65%，顯示國人想要即刻旅行的行動力更加明顯。
隨著端午連假即將到來，萬事達卡自即日起加碼推出包含住宿、交通在內的國旅禮遇，提供持卡人於Agoda、Expedia、Hotels.com、Booking.com等4大訂房網站最低88折起的優惠，而萬事達卡的持卡人不論是搭乘高雄捷運，或是體驗淡海輕軌一日遊，皆能享最低5折起票價，民眾可以輕鬆出行、樂活旅遊。
解封後國旅商機爆發，許多銀行力挺國旅，紛紛祭出刷卡回饋方案，其中北富銀推出飯店訂房優惠，與全台約40家合作飯店，包括陽明山天籟渡假酒店、新竹老爺大酒店、長榮桂冠酒店等，只要刷富邦信用卡訂房最低享5折起優惠，餐飲最低85折起優惠。
台新銀行表示，國內飯店響應政府安心旅遊政策，率先於6月20日至30日端午連假期間，推出國內飯店刷台新信用卡滿6,800即贈1,000旅遊金優惠，可折抵今年底前旅行社、航空、指定訂房網及國內飯店消費，回饋率高達14.7%。
中信銀行則與Agoda、Expedia、Booking.com等3大訂房網合作，即日起至今年底，在Expedia指定專屬訂房網頁，刷中信卡並輸入適用的優惠代碼，享有92折優惠，另在Booking.com部分，刷Visa及Mastercard卡訂房且入住，享稅前房價7%刷卡金回饋，至於Agoda則是7月底前刷中信卡訂房，9月底入住完成，享住宿9折優惠。
另針對偏好團旅出遊的民眾，台新銀行與雄獅、可樂、易遊網及長汎旅行社推出指定行程，享5至10%折扣，達到指定門檻再贈全家便利商店100元禮物卡，含卡片權益最高1.5%回饋，回饋率約11.5%。
北富銀也響應國旅拚經濟，與旅行社推出各項優惠，刷富邦信用卡訂購雄獅旅遊、可樂旅遊指定行程，可享5%優惠；至於訂購易遊網指定商品，滿額也可抽好禮。
此外，喜歡在旅遊票券平台KLOOK、KKday上尋找優惠的民眾，只要刷中信信用

▲迎接國內旅遊商機，銀行信用卡祭高回饋上民眾利上加利。（圖／記者李毓康攝）
記者吳靜君／台北報導
連續多日本土零新冠肺炎（COVID-19）確診案例，許多防疫限制也將在明（7）日大解封，愈來愈多人規劃到戶外旅遊解一解「悶氣」，銀行也共襄盛舉，像是兆豐銀、永豐銀、凱基銀與華南銀等紛紛推出國旅刷卡大回饋，最高回饋最高6%。
兆豐銀行、凱基銀行、永豐銀與華南銀行都推出國內旅遊刷卡優惠，搶一搶國內消費、振興經濟商機。以兆豐銀行來說可享6%的現金回饋最為外界關注，民眾只要在旅行社、飯店等通路，刷兆豐卡刷滿額後可額外獲得6%現金回饋，滿666元再抽刷卡金返現，最多可獲得6萬6666元，回饋幅度相當誘人；另透過雄獅旅遊國內訂房，也不限平假日現折200元，加送全家超商大拿鐵2杯優惠，每季還再抽住宿券。
其他銀行也大方送，像是華南銀行是刷滿2萬元，就送200元的小七抵用券，等於也有1%的回饋；永豐銀則也有刷滿2萬元，送300元，回饋率約1.5%；凱基銀行同樣有誘人回饋，只要滿6000元就送200元，回饋率也有3.3%。
此外，銀行業者也與旅行社合作，例如雄獅旅遊刷兆豐卡可以享受澎湖團體型300元折抵優惠，及全家100元的禮物卡；東南旅遊刷兆豐卡則可以指定主題行程享四人同行現省800元。國泰世華銀則是國內訂房5%的折扣。
▲國內旅遊信用卡回饋。（表／記者吳靜君、資料來源／各銀行，以各銀行公布為準）
銀行業者表示，隨著疫情趨緩，民眾旅遊的興趣也就大增，交通部觀光局也即將推出「安心旅遊」方案，初步規劃自由行每房1000元、團體每人700元的補助方案，所以建議民眾可以研究旅程之外，也可搭配信用卡優惠，讓自己快樂旅遊、開心消費又可享受高回饋、省荷包的小確幸。
----------------------------------------------------------------------

[92] 九成二新鮮人患畢業恐慌症　預估投240封履歷才能錄取一份工作

▲新鮮人預估要投240封履歷才能錄取一份工作。（圖／unsplash）
記者余弦妙／台北報導
畢業季到來，也將是新鮮人求職季的旺季，對此，yes123求職網特地針對「畢業生職場規劃與應徵困境調查」，對於畢業或退伍後求職這件事，合計有92.1%的人透露，其實會「感到恐慌」。
yes123指出，在競爭激烈的畢業求職季，若想殺出重